## Original textures 

In [24]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn as nn
import sys
import matplotlib.pyplot as plt
from tsne_torch import TorchTSNE
import numpy as np
sys.path.append('/home/gridsan/ckoevesdi/.local/lib/python3.9/site-packages/')
from pytorch_metric_learning import losses

sys.path.append('/home/gridsan/ckoevesdi/PooledStatisticsMetamers/poolstatmetamer/')
sys.path.append('/home/gridsan/ckoevesdi/video_pyramids/')
import utils.statnetencoder as sne
import importlib
import imp
import torch.nn.functional as F

#!wget https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py

#sys.path.append(r'C:\Users\chris\Documents\MIT\Statistics_analysis_code\PyTorchSteerablePyramid')
sys.path.append('/home/gridsan/ckoevesdi/PyTorchSteerablePyramid/')
import steerable
import steerable.utils as utils
from steerable.SCFpyr_PyTorch import SCFpyr_PyTorch

torch.manual_seed(16)

#use GPU 2
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [25]:
importlib.reload(sne)
#hyperparams
num_epochs = 100
batch_size = 1
crop_size = 128 
num_stats = 152 
optimizer_type='adam'
#optimizer_type='sgd'
learning_rate = 0.001
num_crops = 5 #changed this to four

multistat_penalty = 0
sparsity_penalty = 0 #0.01
entropic_penalty = 0

statnet_model = sne.StatNetEncoder(img_size=(crop_size,crop_size),
                                   batch_size=batch_size,
                                   num_stats=num_stats,
                                   device=device)
stat_labels = statnet_model.getsstatlabels(device)

Creating solver: pyramid=UBBBBL_6 pooling=WholeImagePooling()


In [26]:
ostat_labels = ['mean', 'variance', 'bandpass_variance 0', 'variance 1', 'skew 1', 'kurtosis 1', 'variance 2', 'skew 2', 'kurtosis 2', 'variance 3', 'skew 3', 'kurtosis 3', 'variance 4', 'skew 4', 'kurtosis 4', 'variance 5', 'skew 5', 'kurtosis 5', 'edge_mean 1 0', 'edge_variance 1 0', 'edge_correlation (1, 2) 0', 'edge_stop 1 0', 'edge_mean 1 1', 'edge_variance 1 1', 'edge_correlation (1, 2) 1', 'edge_stop 1 1', 'edge_mean 1 2', 'edge_variance 1 2', 'edge_correlation (1, 2) 2', 'edge_stop 1 2', 'edge_mean 1 3', 'edge_variance 1 3', 'edge_correlation (1, 2) 3', 'edge_stop 1 3', 'edge_correlation 1 (0, 1)', 'edge_correlation 1 (0, 2)', 'edge_correlation 1 (0, 3)', 'edge_correlation 1 (1, 2)', 'edge_correlation 1 (1, 3)', 'edge_correlation 1 (2, 3)', 'edge_mean 2 0', 'edge_variance 2 0', 'edge_correlation (2, 3) 0', 'edge_stop 2 0', 'edge_mean 2 1', 'edge_variance 2 1', 'edge_correlation (2, 3) 1', 'edge_stop 2 1', 'edge_mean 2 2', 'edge_variance 2 2', 'edge_correlation (2, 3) 2', 'edge_stop 2 2', 'edge_mean 2 3', 'edge_variance 2 3', 'edge_correlation (2, 3) 3', 'edge_stop 2 3', 'edge_correlation 2 (0, 1)', 'edge_correlation 2 (0, 2)', 'edge_correlation 2 (0, 3)', 'edge_correlation 2 (1, 2)', 'edge_correlation 2 (1, 3)', 'edge_correlation 2 (2, 3)', 'edge_mean 3 0', 'edge_variance 3 0', 'edge_correlation (3, 4) 0', 'edge_stop 3 0', 'edge_mean 3 1', 'edge_variance 3 1', 'edge_correlation (3, 4) 1', 'edge_stop 3 1', 'edge_mean 3 2', 'edge_variance 3 2', 'edge_correlation (3, 4) 2', 'edge_stop 3 2', 'edge_mean 3 3', 'edge_variance 3 3', 'edge_correlation (3, 4) 3', 'edge_stop 3 3', 'edge_correlation 3 (0, 1)', 'edge_correlation 3 (0, 2)', 'edge_correlation 3 (0, 3)', 'edge_correlation 3 (1, 2)', 'edge_correlation 3 (1, 3)', 'edge_correlation 3 (2, 3)', 'edge_mean 4 0', 'edge_variance 4 0', 'edge_stop 4 0', 'edge_mean 4 1', 'edge_variance 4 1', 'edge_stop 4 1', 'edge_mean 4 2', 'edge_variance 4 2', 'edge_stop 4 2', 'edge_mean 4 3', 'edge_variance 4 3', 'edge_stop 4 3', 'edge_correlation 4 (0, 1)', 'edge_correlation 4 (0, 2)', 'edge_correlation 4 (0, 3)', 'edge_correlation 4 (1, 2)', 'edge_correlation 4 (1, 3)', 'edge_correlation 4 (2, 3)', 'phase_correlation 1 er (0, 1)', 'phase_correlation 1 er (0, 2)', 'phase_correlation 1 er (0, 3)', 'phase_correlation 1 er (1, 2)', 'phase_correlation 1 er (1, 3)', 'phase_correlation 1 er (2, 3)', 'phase_correlation (1, 2) ei*di 0', 'phase_correlation (1, 2) er*di 0', 'phase_correlation (1, 2) ei*di 1', 'phase_correlation (1, 2) er*di 1', 'phase_correlation (1, 2) ei*di 2', 'phase_correlation (1, 2) er*di 2', 'phase_correlation (1, 2) ei*di 3', 'phase_correlation (1, 2) er*di 3', 'phase_correlation 2 er (0, 1)', 'phase_correlation 2 er (0, 2)', 'phase_correlation 2 er (0, 3)', 'phase_correlation 2 er (1, 2)', 'phase_correlation 2 er (1, 3)', 'phase_correlation 2 er (2, 3)', 'phase_correlation (2, 3) ei*di 0', 'phase_correlation (2, 3) er*di 0', 'phase_correlation (2, 3) ei*di 1', 'phase_correlation (2, 3) er*di 1', 'phase_correlation (2, 3) ei*di 2', 'phase_correlation (2, 3) er*di 2', 'phase_correlation (2, 3) ei*di 3', 'phase_correlation (2, 3) er*di 3', 'phase_correlation 3 er (0, 1)', 'phase_correlation 3 er (0, 2)', 'phase_correlation 3 er (0, 3)', 'phase_correlation 3 er (1, 2)', 'phase_correlation 3 er (1, 3)', 'phase_correlation 3 er (2, 3)', 'phase_correlation (3, 4) ei*di 0', 'phase_correlation (3, 4) er*di 0', 'phase_correlation (3, 4) ei*di 1', 'phase_correlation (3, 4) er*di 1', 'phase_correlation (3, 4) ei*di 2', 'phase_correlation (3, 4) er*di 2', 'phase_correlation (3, 4) ei*di 3', 'phase_correlation (3, 4) er*di 3', 'phase_correlation 4 er (0, 1)', 'phase_correlation 4 er (0, 2)', 'phase_correlation 4 er (0, 3)', 'phase_correlation 4 er (1, 2)', 'phase_correlation 4 er (1, 3)', 'phase_correlation 4 er (2, 3)']

In [27]:
ostat_labels = stat_labels

In [28]:
from torch.utils.data import Dataset
from PIL import Image
import os

class KTHImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        
        # Filtering only directories for classes
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for sample_folder in os.listdir(class_dir):
                sample_folder_path = os.path.join(class_dir, sample_folder)
                for filename in os.listdir(sample_folder_path):
                    img_path = os.path.join(sample_folder_path, filename)
                    self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Specify transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.Grayscale(),
    transforms.ToTensor(),  
    transforms.ConvertImageDtype(torch.float32)
])

# Initialize dataset and dataloader
sys.path.append('/home/gridsan/ckoevesdi/data_KTH/KTH-TIPS2-b/')

root_dir = '/home/gridsan/ckoevesdi/data_KTH/KTH-TIPS2-b/'
dataset = KTHImageDataset(root_dir=root_dir, transform=transform)
brucy_dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [29]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class KTHImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        
        # Filtering only directories for classes
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for sample_folder in os.listdir(class_dir):
                sample_folder_path = os.path.join(class_dir, sample_folder)
                for filename in os.listdir(sample_folder_path):
                    img_path = os.path.join(sample_folder_path, filename)
                    self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Open the image directly in grayscale mode
        image = Image.open(img_path).convert("L")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Specify transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),  # Resize the image to your desired dimensions
    transforms.ToTensor(),  # Convert to a tensor
])
# Initialize dataset and dataloader
sys.path.append('/home/gridsan/ckoevesdi/data_KTH/KTH-TIPS2-b/')

root_dir = '/home/gridsan/ckoevesdi/data_KTH/KTH-TIPS2-b/'
dataset = KTHImageDataset(root_dir=root_dir, transform=transform)
brucy_dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [30]:
import utils.brucenet as bn
brucy = bn.BruceNet(pooling_region_size=1e20, pyramid_params=False, dummy_img = torch.zeros(20,
                                     1,
                                     200,
                                     200)).to(device)

Creating solver: pyramid=UBBBBL_6 pooling=WholeImagePooling()


In [31]:
all_statisticsOT = []

for j, (texture_batch, labels) in enumerate(brucy_dataloader):
    # Assuming texture_batch has a shape of [20, 5, 1, 128, 128]
    #print(texture_batch.shape)
    output = texture_batch.to(device)
    output = torch.stack([output, output]).squeeze(1)
    #print(output.shape)
    #print(labels)
    statistics = brucy(output)  # This should output a tensor of shape [2, 150]
    statistics = statistics[0,:]
    #print(statistics[0,:])
    # You can now store these statistics, along with the label and original image index
    all_statisticsOT.append({
        'statistics': statistics.cpu().numpy(),
        #'label': labels[i].item(),
        'original_image_index': labels.cpu().numpy()
    })

In [37]:
statistics_arrayOT = np.array([item['statistics'] for item in all_statisticsOT])
print(statistics_arrayOT.shape)
print(statistics_arrayOT[:,2])
print(statistics_arrayHB[:,2])


(4752, 152)
[0.12119011 0.11278209 0.1403659  ... 0.02575193 0.03227771 0.03183674]
[0.12099025 0.11259612 0.11539846 ... 0.00908253 0.03178425 0.0829745 ]


## Heeger and Bergen

In [42]:
class KTHImageDatasetHB(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Filtering only directories for classes
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)

            # Scanning for image files directly in the class folder
            for filename in os.listdir(class_dir):
                img_path = os.path.join(class_dir, filename)

                if os.path.isfile(img_path):  # Checking if it's a file
                    self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


# Specify transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.Grayscale(),
    transforms.ToTensor(),  
    transforms.ConvertImageDtype(torch.float32)
])

# Initialize dataset and dataloader
sys.path.append('/home/gridsan/ckoevesdi/data_KTH_HB/')

root_dir = '/home/gridsan/ckoevesdi/data_KTH_HB/'
dataset = KTHImageDatasetHB(root_dir=root_dir, transform=transform)
brucy_dataloaderHB = DataLoader(dataset, batch_size=1, shuffle=False)

In [47]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class KTHImageDatasetHB(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Filtering only directories for classes
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)

            # Scanning for image files directly in the class folder
            for filename in os.listdir(class_dir):
                img_path = os.path.join(class_dir, filename)

                if os.path.isfile(img_path):  # Checking if it's a file
                    self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Open the image directly in grayscale mode
        image = Image.open(img_path).convert("L")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Specify transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),  # Resize the image to your desired dimensions
    transforms.ToTensor(),  # Convert to a tensor
])
# Initialize dataset and dataloader
sys.path.append('/home/gridsan/ckoevesdi/data_KTH_HB/')

root_dir = '/home/gridsan/ckoevesdi/data_KTH_HB/'
dataset = KTHImageDatasetHB(root_dir=root_dir, transform=transform)
brucy_dataloaderHB = DataLoader(dataset, batch_size=1, shuffle=False)

In [48]:
print(len(dataset))

4752


In [49]:
all_statisticsHB = []

for j, (texture_batch, labels) in enumerate(brucy_dataloaderHB):
    # Assuming texture_batch has a shape of [20, 5, 1, 128, 128]
    #print(texture_batch.shape)
    output = texture_batch.to(device)
    output = torch.stack([output, output]).squeeze(1)
    #print(output.shape)
    #print(labels)
    statistics = brucy(output)  # This should output a tensor of shape [2, 150]
    statistics = statistics[0,:]
    #print(statistics[0,:])
    # You can now store these statistics, along with the label and original image index
    all_statisticsHB.append({
        'statistics': statistics.cpu().numpy(),
        #'label': labels[i].item(),
        'original_image_index': labels.cpu().numpy()
    })

In [50]:
statistics_arrayHB = np.array([item['statistics'] for item in all_statisticsHB])
print(statistics_arrayHB.shape)

(4752, 152)


## Analysis

In [51]:
relative_errors = np.mean(np.abs(statistics_arrayOT - statistics_arrayHB) / (np.abs(statistics_arrayOT)), axis=0)

In [52]:
# Get the indices that would sort the array
sorted_indices = np.argsort(relative_errors)
# Assuming you have arrays relative_errors and sorted_indices
np.save('relative_errors.npy', relative_errors)
np.save('sorted_indices.npy', sorted_indices)
# Print the ranked feature indices
print("Ranked feature indices from best to worst:")
print(sorted_indices)

Ranked feature indices from best to worst:
[  0  86  92  89  95  72  64  68   1  76   5   8  11  66  14  50  42  17
  87  46  54  74 146  93 148  98 100  24 101 149  28 103 151 102  20  99
  32  90  52  44  73  70  65  96  78  97  91 135 137 134  83 132  85  30
  71  80  51  82  79  22  84   2   6  43  48 121  81  56   9 123  69  77
  61 118 120  63  12  29  58   4  60  15  26  62  94  18  34 107  49 109
  21  47  55  39  59  57 104  88  41  36 106  40  38  25  33  37   3  27
  35   7 115  10 129  75  13  67  16 111  19 127  31  45  53  23 110 139
 131 113 125 114 124 130 116 117 128 142 143 112 141 147 140 126 138 144
 145 122 133 108 150 136 119 105]


In [53]:
print("Ranked features from best to worst:")
i = 1
for idx in sorted_indices:
    i = i + 1
    print(f"Rank {i}: {ostat_labels[idx]}, Relative Error: {relative_errors[idx]:.4f}")

Ranked features from best to worst:
Rank 2:  mean Base, Relative Error: 0.1750
Rank 3:  edge_mean L4 Ori0, Relative Error: 0.2290
Rank 4:  edge_mean L4 Ori2, Relative Error: 0.2368
Rank 5:  edge_mean L4 Ori1, Relative Error: 0.2560
Rank 6:  edge_mean L4 Ori3, Relative Error: 0.2694
Rank 7:  edge_mean L3 Ori2, Relative Error: 0.3235
Rank 8:  edge_mean L3 Ori0, Relative Error: 0.3486
Rank 9:  edge_mean L3 Ori1, Relative Error: 0.3865
Rank 10:  variance Base, Relative Error: 0.3875
Rank 11:  edge_mean L3 Ori3, Relative Error: 0.3883
Rank 12:  variance L1, Relative Error: 0.3911
Rank 13:  variance L2, Relative Error: 0.4010
Rank 14:  variance L3, Relative Error: 0.4129
Rank 15:  edge_correlation L3_4 Ori0, Relative Error: 0.4196
Rank 16:  variance L4, Relative Error: 0.4241
Rank 17:  edge_mean L2 Ori2, Relative Error: 0.4261
Rank 18:  edge_mean L2 Ori0, Relative Error: 0.4327
Rank 19:  variance L5, Relative Error: 0.4335
Rank 20:  edge_variance L4 Ori0, Relative Error: 0.4436
Rank 21:  edg

In [13]:
import numpy as np

# Assuming 2D arrays for your datasets
combined_data = np.vstack([statistics_arrayHB, statistics_arrayOT])

# Calculate mean and standard deviation for the combined data
mean_combined = np.mean(combined_data, axis=0)
std_combined = np.std(combined_data, axis=0)

# Normalize both datasets using the combined mean and standard deviation
normalized_HB = (statistics_arrayHB - mean_combined) / std_combined
normalized_OT = (statistics_arrayOT - mean_combined) / std_combined


In [14]:
relative_errors = np.mean(np.abs(normalized_OT - normalized_HB), axis=0)

In [15]:
# Get the indices that would sort the array
sorted_indices = np.argsort(relative_errors)

# Print the ranked feature indices
print("Ranked feature indices from best to worst:")
print(sorted_indices)

Ranked feature indices from best to worst:
[ 69  77  73  88 139  92  86  68 143  65 100  94  76 129 125  99  67  97
  89  96 133  75  47  81 101  98  55  95  82  83  46  51 147 135 127  87
  54 144 146  71  80  78 149  79 132  43  93  53  50 119 121  45 130  24
  32 123  59  61 108  28  72  49 107 113  60  27 105  37  23  29 141  39
  63  31 115 145 111  57  58  64 122  38  25  21 118   2  56  35  91 131
  33  85  42 116  34  90 102  66  36 137  74  84  41  20  19 104  70 109
 110 134  22 106  52  44 124  18 117  30 138 114  62 103 142 112  48  40
  26 120 128 148   0  15 136  12   9   5   6   3 126   1  17  16  14  13
   8  11  10   4   7 140]


In [16]:
print("Ranked features from best to worst:")
i = 1
for idx in sorted_indices:
    i = i + 1
    print(f"Rank {i}: {ostat_labels[idx]}, Relative Error: {relative_errors[idx]:.4f}")

Ranked features from best to worst:
Rank 2: edge_stop 3 1, Relative Error: 0.3752
Rank 3: edge_stop 3 3, Relative Error: 0.3947
Rank 4: edge_stop 3 2, Relative Error: 0.4015
Rank 5: edge_variance 4 1, Relative Error: 0.4069
Rank 6: phase_correlation (3, 4) er*di 1, Relative Error: 0.4121
Rank 7: edge_stop 4 2, Relative Error: 0.4141
Rank 8: edge_stop 4 0, Relative Error: 0.4173
Rank 9: edge_correlation (3, 4) 1, Relative Error: 0.4177
Rank 10: phase_correlation (3, 4) er*di 3, Relative Error: 0.4225
Rank 11: edge_stop 3 0, Relative Error: 0.4328
Rank 12: edge_correlation 4 (1, 3), Relative Error: 0.4342
Rank 13: edge_variance 4 3, Relative Error: 0.4371
Rank 14: edge_correlation (3, 4) 3, Relative Error: 0.4421
Rank 15: phase_correlation (2, 3) er*di 3, Relative Error: 0.4615
Rank 16: phase_correlation (2, 3) er*di 1, Relative Error: 0.4626
Rank 17: edge_correlation 4 (1, 2), Relative Error: 0.4627
Rank 18: edge_variance 3 1, Relative Error: 0.4645
Rank 19: edge_correlation 4 (0, 2), R